In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
df = pd.read_csv(url, sep=';', decimal=',', parse_dates={'Datetime': ['Date', 'Time']})
df = df.iloc[:, :15].dropna()  # Drop any rows with NaN values

data = df[['CO(GT)']].values
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['AirQualityUCI.csv', 'AirQualityUCI.xlsx']

In [ ]:
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        X.append(dataset[i:(i+look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

In [ ]:
look_back = 24  # Use last 24 hours for predicting next value
X, Y = create_dataset(data_scaled, look_back)
X = X.reshape((X.shape[0], X.shape[1], 1))

In [3]:
def build_rnn():
    model = Sequential()
    model.add(LSTM(50, input_shape=(look_back, 1)))
    model.add(Dense(1))
    return model

In [4]:
rnn_model = build_rnn()
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

start_time = time.time()
rnn_model.fit(X, Y, epochs=10, batch_size=64, validation_split=0.2, verbose=1)
rnn_training_time = time.time() - start_time

rnn_train_loss = rnn_model.evaluate(X, Y, verbose=2)
print(f"RNN Training Loss (MSE): {rnn_train_loss:.4f}, Training Time: {rnn_training_time:.2f} seconds")

NameError: name 'look_back' is not defined